In [2]:
# data manipulation libraries
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.model_selection import train_test_split

# pyspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import types, functions as F

# random seed for reproducibility
RANDOM_SEED = 15

import shap
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
from sklearn.model_selection import RepeatedKFold
plt.style.use('seaborn-white')

app_name = 'poi-reseller'
conf = (SparkConf()
        .setMaster("yarn-client")
        .setAppName(app_name)
        .set("spark.executor.memory", "8g")
        .set("spark.executor.instances", "1")
        .set("spark.executor.cores", "4")
        .set("spark.driver.memory", "8g")
        .set("spark.yarn.queue", "root.hue_dmp")
        .set("spark.default.parallelism", "8")
        .set("spark.sql.shuffle.partitions", "1000")
        .set("spark.shuffle.service.enabled", "true")
        .set("spark.dynamicAllocation.enabled", "true")
        .set("spark.yarn.driver.memoryOverhead", "4096")
        .set("spark.yarn.executor.memoryOverhead", "4096")
        .set("spark.kryoserializer.buffer.max", "1g")
        .set("spark.dynamicAllocation.minExecutors", "1")
        .set("spark.dynamicAllocation.maxExecutors", "20")
        .set("spark.driver.maxResultSize", "4g")
        .set("spark.sql.crossJoin.enabled", "true")
        .set("spark.hadoop.fs.permissions.umask-mode", "002")
        .set("spark.sql.autoBroadcastJoinThreshold", "-1")
        .set("spark.dynamicAllocation.initialExecutors", "1"))
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
spark

In [26]:
df_csv = pd.read_csv('/home/cdsw/_SELECT_x_c_hvc_rfm_segmentation_FROM_SELECT_b_msisdn_a_FROM_mck_202107222059.csv')
df_csv.head()

,x.msisdn,x.hash_msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_health_care_facility,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_community_facility,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_pharmacy,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport,x.fea_outlet_under_100m_to_consumer_services,x.fea_outlet_under_250m_to_consumer_services,x.fea_outlet_under_500m_to_consumer_services,x.fea_outlet_under_1000m_to_consumer_services,x.fea_outlet_under_100m_to_restaurant,x.fea_outlet_under_250m_to_restaurant,x.fea_outlet_under_500m_to_restaurant,x.fea_outlet_under_1000m_to_restaurant,x.fea_outlet_under_100m_to_convenience_store,x.fea_outlet_under_250m_to_convenience_store,x.fea_outlet_under_500m_to_convenience_store,x.fea_outlet_under_1000m_to_convenience_store,x.fea_outlet_under_100m_to_education_facility,x.fea_outlet_under_250m_to_education_facility,x.fea_outlet_under_500m_to_education_facility,x.fea_outlet_under_1000m_to_education_facility,x.fea_outlet_under_100m_to_consumer_goods,x.fea_outlet_under_250m_to_consumer_goods,x.fea_outlet_under_500m_to_consumer_goods,x.fea_outlet_under_1000m_to_consumer_goods,x.fea_outlet_under_100m_to_hospital_or_health_care_facility,x.fea_outlet_under_250m_to_hospital_or_health_care_facility,x.fea_outlet_under_500m_to_hospital_or_health_care_facility,x.fea_outlet_under_1000m_to_hospital_or_health_care_facility,x.fea_outlet_under_100m_to_religious_place,x.fea_outlet_under_250m_to_religious_place,x.fea_outlet_under_500m_to_religious_place,x.fea_outlet_under_1000m_to_religious_place,x.fea_outlet_under_100m_to_food_and_drink,x.fea_outlet_under_250m_to_food_and_drink,x.fea_outlet_under_500m_to_food_and_drink,x.fea_outlet_under_1000m_to_food_and_drink,x.fea_outlet_under_100m_to_government_or_community_facility,x.fea_outlet_under_250m_to_government_or_community_facility,x.fea_outlet_under_500m_to_government_or_community_facility,x.fea_outlet_under_1000m_to_government_or_community_facility,x.fea_outlet_under_100m_to_business_industry,x.fea_outlet_under_250m_to_business_industry,x.fea_outlet_under_500m_to_business_industry,x.fea_outlet_under_1000m_to_business_industry,x.fea_outlet_under_100m_to_city_town_or_village,x.fea_outlet_under_250m_to_city_town_or_village,x.fea_outlet_under_500m_to_city_town_or_village,x.fea_outlet_under_1000m_to_city_town_or_village,x.fea_outlet_under_100m_to_drugstore_or_pharmacy,x.fea_outlet_under_250m_to_drugstore_or_pharmacy,x.fea_outlet_under_500m_to_drugstore_or_pharmacy,x.fea_outlet_under_1000m_to_drugstore_or_pharmacy,x.fea_outlet_under_100m_to_building,x.fea_outlet_under_250m_to_building,x.fea_outlet_under_500m_to_building,x.fea_outlet_under_1000m_to_building,x.fea_outlet_under_100m_to_facilities,x.fea_outlet_under_250m_to_facilities,x.fea_outlet_under_500m_to_facilities,x.fea_outlet_under_1000m_to_facilities,x.fea_outlet_under_100m_to_airport,x.fea_outlet_under_250m_to_airport,x.fea_outlet_under_500m_to_airport,x.fea_outlet_under_1000m_to_airport,c.hvc_rfm_segmentation
0,6281312303877,36533d260394fe678d3b443092fe7632,14.82,37.75,33.22,63.25,31.45,60.66,247.04,226.19,243.42,37.31,763.28,25.11,368.48,592.07,0.00,14,22,75,352,16,34,81,439,8,10,23,174,3,6,29,122,1,1,9,88,2,5,18,104,0,1,12,64,0,2,6,72,0,1,5,23,1,12,29,93,0,0,0,1,1,3,4,16,0,0,2,9,0,0,0,4,0,0,0,0,high
1,62811693455,f5ebc71dbb5f3df568cfa41cb7c44de6,56.98,33.80,221.93,57.75,75.57,95.51,27.70,232.78,224.88,107.62,422.34,198.26,361.61,750.41,0.00,1,9,86,516,2,16,100,582,0,2,19,130,1,7,31,191,1,3,7,113,1,7,33,229,1,12,26,116,0,2,5,64,0,1,21,48,0,1,11,7

In [27]:
df.columns

Index(['x.msisdn', 'x.fea_min_distance_outlet_to_consumer_services',
       'x.fea_min_distance_outlet_to_restaurant',
       'x.fea_min_distance_outlet_to_convenience_store',
       'x.fea_min_distance_outlet_to_education_facility',
       'x.fea_min_distance_outlet_to_consumer_goods',
       'x.fea_min_distance_outlet_to_hospital_or_health_care_facility',
       'x.fea_min_distance_outlet_to_religious_place',
       'x.fea_min_distance_outlet_to_food_and_drink',
       'x.fea_min_distance_outlet_to_government_or_community_facility',
       'x.fea_min_distance_outlet_to_business_industry',
       'x.fea_min_distance_outlet_to_city_town_or_village',
       'x.fea_min_distance_outlet_to_drugstore_or_pharmacy',
       'x.fea_min_distance_outlet_to_building',
       'x.fea_min_distance_outlet_to_facilities',
       'x.fea_min_distance_outlet_to_airport'],
      dtype='object')

In [8]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

df = spark.sql("""select 
                        'x.msisdn',
                        'x.fea_min_distance_outlet_to_consumer_services',
                        'x.fea_min_distance_outlet_to_restaurant',
                        'x.fea_min_distance_outlet_to_convenience_store',
                        'x.fea_min_distance_outlet_to_education_facility',
                        'x.fea_min_distance_outlet_to_consumer_goods',
                        'x.fea_min_distance_outlet_to_hospital_or_health_care_facility',
                        'x.fea_min_distance_outlet_to_religious_place',
                        'x.fea_min_distance_outlet_to_food_and_drink',
                        'x.fea_min_distance_outlet_to_government_or_community_facility',
                        'x.fea_min_distance_outlet_to_business_industry',
                        'x.fea_min_distance_outlet_to_city_town_or_village',
                        'x.fea_min_distance_outlet_to_drugstore_or_pharmacy',
                        'x.fea_min_distance_outlet_to_building',
                        'x.fea_min_distance_outlet_to_facilities',
                        'x.fea_min_distance_outlet_to_airport'
                    from
                        dbi.join_fiveg_msisdn_to_hash 
""").toPandas()
df.head()

,x.msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_health_care_facility,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_community_facility,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_pharmacy,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport
0,x.msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_healt...,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_com...,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_phar...,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport
1,x.msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_healt...,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_com...,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_phar...,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport
2,x.msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_healt...,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_com...,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_phar...,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport
3,x.msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_healt...,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_com...,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_phar...,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport
4,x.msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_healt...,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_com...,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_dis

In [28]:
df_csv['c.hvc_rfm_segmentation'].replace(to_replace=['null', 'low', 'med', 'high'], value=[0, 1, 2, 3], inplace=True)

In [29]:
df_csv.head()

,x.msisdn,x.hash_msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_health_care_facility,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_community_facility,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_pharmacy,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport,x.fea_outlet_under_100m_to_consumer_services,x.fea_outlet_under_250m_to_consumer_services,x.fea_outlet_under_500m_to_consumer_services,x.fea_outlet_under_1000m_to_consumer_services,x.fea_outlet_under_100m_to_restaurant,x.fea_outlet_under_250m_to_restaurant,x.fea_outlet_under_500m_to_restaurant,x.fea_outlet_under_1000m_to_restaurant,x.fea_outlet_under_100m_to_convenience_store,x.fea_outlet_under_250m_to_convenience_store,x.fea_outlet_under_500m_to_convenience_store,x.fea_outlet_under_1000m_to_convenience_store,x.fea_outlet_under_100m_to_education_facility,x.fea_outlet_under_250m_to_education_facility,x.fea_outlet_under_500m_to_education_facility,x.fea_outlet_under_1000m_to_education_facility,x.fea_outlet_under_100m_to_consumer_goods,x.fea_outlet_under_250m_to_consumer_goods,x.fea_outlet_under_500m_to_consumer_goods,x.fea_outlet_under_1000m_to_consumer_goods,x.fea_outlet_under_100m_to_hospital_or_health_care_facility,x.fea_outlet_under_250m_to_hospital_or_health_care_facility,x.fea_outlet_under_500m_to_hospital_or_health_care_facility,x.fea_outlet_under_1000m_to_hospital_or_health_care_facility,x.fea_outlet_under_100m_to_religious_place,x.fea_outlet_under_250m_to_religious_place,x.fea_outlet_under_500m_to_religious_place,x.fea_outlet_under_1000m_to_religious_place,x.fea_outlet_under_100m_to_food_and_drink,x.fea_outlet_under_250m_to_food_and_drink,x.fea_outlet_under_500m_to_food_and_drink,x.fea_outlet_under_1000m_to_food_and_drink,x.fea_outlet_under_100m_to_government_or_community_facility,x.fea_outlet_under_250m_to_government_or_community_facility,x.fea_outlet_under_500m_to_government_or_community_facility,x.fea_outlet_under_1000m_to_government_or_community_facility,x.fea_outlet_under_100m_to_business_industry,x.fea_outlet_under_250m_to_business_industry,x.fea_outlet_under_500m_to_business_industry,x.fea_outlet_under_1000m_to_business_industry,x.fea_outlet_under_100m_to_city_town_or_village,x.fea_outlet_under_250m_to_city_town_or_village,x.fea_outlet_under_500m_to_city_town_or_village,x.fea_outlet_under_1000m_to_city_town_or_village,x.fea_outlet_under_100m_to_drugstore_or_pharmacy,x.fea_outlet_under_250m_to_drugstore_or_pharmacy,x.fea_outlet_under_500m_to_drugstore_or_pharmacy,x.fea_outlet_under_1000m_to_drugstore_or_pharmacy,x.fea_outlet_under_100m_to_building,x.fea_outlet_under_250m_to_building,x.fea_outlet_under_500m_to_building,x.fea_outlet_under_1000m_to_building,x.fea_outlet_under_100m_to_facilities,x.fea_outlet_under_250m_to_facilities,x.fea_outlet_under_500m_to_facilities,x.fea_outlet_under_1000m_to_facilities,x.fea_outlet_under_100m_to_airport,x.fea_outlet_under_250m_to_airport,x.fea_outlet_under_500m_to_airport,x.fea_outlet_under_1000m_to_airport,c.hvc_rfm_segmentation
0,6281312303877,36533d260394fe678d3b443092fe7632,14.82,37.75,33.22,63.25,31.45,60.66,247.04,226.19,243.42,37.31,763.28,25.11,368.48,592.07,0.00,14,22,75,352,16,34,81,439,8,10,23,174,3,6,29,122,1,1,9,88,2,5,18,104,0,1,12,64,0,2,6,72,0,1,5,23,1,12,29,93,0,0,0,1,1,3,4,16,0,0,2,9,0,0,0,4,0,0,0,0,3.0
1,62811693455,f5ebc71dbb5f3df568cfa41cb7c44de6,56.98,33.80,221.93,57.75,75.57,95.51,27.70,232.78,224.88,107.62,422.34,198.26,361.61,750.41,0.00,1,9,86,516,2,16,100,582,0,2,19,130,1,7,31,191,1,3,7,113,1,7,33,229,1,12,26,116,0,2,5,64,0,1,21,48,0,1,11,75

In [33]:
df_csv.isnull().sum(axis = 0)

x.msisdn                                                         0
x.hash_msisdn                                                    0
x.fea_min_distance_outlet_to_consumer_services                   0
x.fea_min_distance_outlet_to_restaurant                          0
x.fea_min_distance_outlet_to_convenience_store                   0
x.fea_min_distance_outlet_to_education_facility                  0
x.fea_min_distance_outlet_to_consumer_goods                      0
x.fea_min_distance_outlet_to_hospital_or_health_care_facility    0
x.fea_min_distance_outlet_to_religious_place                     0
x.fea_min_distance_outlet_to_food_and_drink                      0
x.fea_min_distance_outlet_to_government_or_community_facility    0
x.fea_min_distance_outlet_to_business_industry                   0
x.fea_min_distance_outlet_to_city_town_or_village                0
x.fea_min_distance_outlet_to_drugstore_or_pharmacy               0
x.fea_min_distance_outlet_to_building                         

In [32]:
df_csv['c.hvc_rfm_segmentation'].fillna(0, inplace=True)

In [34]:
dist_feature = ['x.msisdn',
       'x.fea_min_distance_outlet_to_consumer_services',
       'x.fea_min_distance_outlet_to_restaurant',
       'x.fea_min_distance_outlet_to_convenience_store',
       'x.fea_min_distance_outlet_to_education_facility',
       'x.fea_min_distance_outlet_to_consumer_goods',
       'x.fea_min_distance_outlet_to_hospital_or_health_care_facility',
       'x.fea_min_distance_outlet_to_religious_place',
       'x.fea_min_distance_outlet_to_food_and_drink',
       'x.fea_min_distance_outlet_to_government_or_community_facility',
       'x.fea_min_distance_outlet_to_business_industry',
       'x.fea_min_distance_outlet_to_city_town_or_village',
       'x.fea_min_distance_outlet_to_drugstore_or_pharmacy',
       'x.fea_min_distance_outlet_to_building',
       'x.fea_min_distance_outlet_to_facilities',
       'x.fea_min_distance_outlet_to_airport','c.hvc_rfm_segmentation']

for cols in dist_feature:
    df_csv[cols]=df_csv[cols].astype(float)

df_csv.head()

,x.msisdn,x.hash_msisdn,x.fea_min_distance_outlet_to_consumer_services,x.fea_min_distance_outlet_to_restaurant,x.fea_min_distance_outlet_to_convenience_store,x.fea_min_distance_outlet_to_education_facility,x.fea_min_distance_outlet_to_consumer_goods,x.fea_min_distance_outlet_to_hospital_or_health_care_facility,x.fea_min_distance_outlet_to_religious_place,x.fea_min_distance_outlet_to_food_and_drink,x.fea_min_distance_outlet_to_government_or_community_facility,x.fea_min_distance_outlet_to_business_industry,x.fea_min_distance_outlet_to_city_town_or_village,x.fea_min_distance_outlet_to_drugstore_or_pharmacy,x.fea_min_distance_outlet_to_building,x.fea_min_distance_outlet_to_facilities,x.fea_min_distance_outlet_to_airport,x.fea_outlet_under_100m_to_consumer_services,x.fea_outlet_under_250m_to_consumer_services,x.fea_outlet_under_500m_to_consumer_services,x.fea_outlet_under_1000m_to_consumer_services,x.fea_outlet_under_100m_to_restaurant,x.fea_outlet_under_250m_to_restaurant,x.fea_outlet_under_500m_to_restaurant,x.fea_outlet_under_1000m_to_restaurant,x.fea_outlet_under_100m_to_convenience_store,x.fea_outlet_under_250m_to_convenience_store,x.fea_outlet_under_500m_to_convenience_store,x.fea_outlet_under_1000m_to_convenience_store,x.fea_outlet_under_100m_to_education_facility,x.fea_outlet_under_250m_to_education_facility,x.fea_outlet_under_500m_to_education_facility,x.fea_outlet_under_1000m_to_education_facility,x.fea_outlet_under_100m_to_consumer_goods,x.fea_outlet_under_250m_to_consumer_goods,x.fea_outlet_under_500m_to_consumer_goods,x.fea_outlet_under_1000m_to_consumer_goods,x.fea_outlet_under_100m_to_hospital_or_health_care_facility,x.fea_outlet_under_250m_to_hospital_or_health_care_facility,x.fea_outlet_under_500m_to_hospital_or_health_care_facility,x.fea_outlet_under_1000m_to_hospital_or_health_care_facility,x.fea_outlet_under_100m_to_religious_place,x.fea_outlet_under_250m_to_religious_place,x.fea_outlet_under_500m_to_religious_place,x.fea_outlet_under_1000m_to_religious_place,x.fea_outlet_under_100m_to_food_and_drink,x.fea_outlet_under_250m_to_food_and_drink,x.fea_outlet_under_500m_to_food_and_drink,x.fea_outlet_under_1000m_to_food_and_drink,x.fea_outlet_under_100m_to_government_or_community_facility,x.fea_outlet_under_250m_to_government_or_community_facility,x.fea_outlet_under_500m_to_government_or_community_facility,x.fea_outlet_under_1000m_to_government_or_community_facility,x.fea_outlet_under_100m_to_business_industry,x.fea_outlet_under_250m_to_business_industry,x.fea_outlet_under_500m_to_business_industry,x.fea_outlet_under_1000m_to_business_industry,x.fea_outlet_under_100m_to_city_town_or_village,x.fea_outlet_under_250m_to_city_town_or_village,x.fea_outlet_under_500m_to_city_town_or_village,x.fea_outlet_under_1000m_to_city_town_or_village,x.fea_outlet_under_100m_to_drugstore_or_pharmacy,x.fea_outlet_under_250m_to_drugstore_or_pharmacy,x.fea_outlet_under_500m_to_drugstore_or_pharmacy,x.fea_outlet_under_1000m_to_drugstore_or_pharmacy,x.fea_outlet_under_100m_to_building,x.fea_outlet_under_250m_to_building,x.fea_outlet_under_500m_to_building,x.fea_outlet_under_1000m_to_building,x.fea_outlet_under_100m_to_facilities,x.fea_outlet_under_250m_to_facilities,x.fea_outlet_under_500m_to_facilities,x.fea_outlet_under_1000m_to_facilities,x.fea_outlet_under_100m_to_airport,x.fea_outlet_under_250m_to_airport,x.fea_outlet_under_500m_to_airport,x.fea_outlet_under_1000m_to_airport,c.hvc_rfm_segmentation
0,6.281312e+12,36533d260394fe678d3b443092fe7632,14.82,37.75,33.22,63.25,31.45,60.66,247.04,226.19,243.42,37.31,763.28,25.11,368.48,592.07,0.00,14,22,75,352,16,34,81,439,8,10,23,174,3,6,29,122,1,1,9,88,2,5,18,104,0,1,12,64,0,2,6,72,0,1,5,23,1,12,29,93,0,0,0,1,1,3,4,16,0,0,2,9,0,0,0,4,0,0,0,0,3.0
1,6.281169e+10,f5ebc71dbb5f3df568cfa41cb7c44de6,56.98,33.80,221.93,57.75,75.57,95.51,27.70,232.78,224.88,107.62,422.34,198.26,361.61,750.41,0.00,1,9,86,516,2,16,100,582,0,2,19,130,1,7,31,191,1,3,7,113,1,7,33,229,1,12,26,116,0,2,5,64,0,1,21,48,0,1,11,75

In [4]:
df.dtypes

x.msisdn                                            int64
x.hash_msisdn                                      object
x.fea_min_distance_outlet_to_consumer_services    float64
x.fea_min_distance_outlet_to_restaurant           float64
x.fea_min_distance_outlet_to_convenience_store    float64
                                                   ...   
x.fea_outlet_under_100m_to_airport                  int64
x.fea_outlet_under_250m_to_airport                  int64
x.fea_outlet_under_500m_to_airport                  int64
x.fea_outlet_under_1000m_to_airport                 int64
c.hvc_rfm_segmentation                             object
Length: 78, dtype: object

In [35]:
print (df_csv[[
       'x.fea_min_distance_outlet_to_consumer_services',
       'x.fea_min_distance_outlet_to_restaurant',
       'x.fea_min_distance_outlet_to_convenience_store',
       'x.fea_min_distance_outlet_to_education_facility',
       'x.fea_min_distance_outlet_to_consumer_goods',
       'x.fea_min_distance_outlet_to_hospital_or_health_care_facility',
       'x.fea_min_distance_outlet_to_religious_place',
       'x.fea_min_distance_outlet_to_food_and_drink',
       'x.fea_min_distance_outlet_to_government_or_community_facility',
       'x.fea_min_distance_outlet_to_business_industry',
       'x.fea_min_distance_outlet_to_city_town_or_village',
       'x.fea_min_distance_outlet_to_drugstore_or_pharmacy',
       'x.fea_min_distance_outlet_to_building',
       'x.fea_min_distance_outlet_to_facilities',
       'x.fea_min_distance_outlet_to_airport'
          ]].corrwith(df_csv['c.hvc_rfm_segmentation']))

x.fea_min_distance_outlet_to_consumer_services                  -0.026197
x.fea_min_distance_outlet_to_restaurant                          0.009652
x.fea_min_distance_outlet_to_convenience_store                   0.008994
x.fea_min_distance_outlet_to_education_facility                 -0.023879
x.fea_min_distance_outlet_to_consumer_goods                     -0.059633
x.fea_min_distance_outlet_to_hospital_or_health_care_facility   -0.012337
x.fea_min_distance_outlet_to_religious_place                     0.013673
x.fea_min_distance_outlet_to_food_and_drink                     -0.007417
x.fea_min_distance_outlet_to_government_or_community_facility   -0.010048
x.fea_min_distance_outlet_to_business_industry                   0.003652
x.fea_min_distance_outlet_to_city_town_or_village                0.066486
x.fea_min_distance_outlet_to_drugstore_or_pharmacy               0.005020
x.fea_min_distance_outlet_to_building                            0.041837
x.fea_min_distance_outlet_to_facilitie

In [36]:
X           = df_csv.drop(['x.msisdn'], axis=1)
y           = df_csv[['c.hvc_rfm_segmentation']]
#y           = np.log(y['c.hvc_rfm_segmentation']+1)
model       = xgb.XGBRegressor().fit(X, y)
explainer   = shap.Explainer(model)
shap_values = explainer(X)
shap.plots.beeswarm(shap_values, max_display=20)

ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.x.hash_msisdn

In [6]:
from numpy import absolute
model       = xgb.XGBRegressor().fit(X, y)
cv          = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores      = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
scores      = absolute(scores)
print('Mean r2: %.1f (%.1f)' % (scores.mean(), scores.std()) )

Mean r2: 0.6 (0.2)


In [9]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, y)

RandomForestRegressor()

In [10]:
sklearn.metrics.SCORERS.keys()

NameError: name 'sklearn' is not defined

In [11]:
y.median()

11.193799247026783

In [12]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, y)

LinearRegression()

_________________________________________________

In [13]:
model = RandomForestRegressor()
model.fit(X, y)

RandomForestRegressor()

In [16]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X, y)

DecisionTreeRegressor()